In [1]:
import giskard
from typing import Sequence, Optional
from giskard.llm.client import set_default_client
from giskard.llm.client.base import LLMClient, ChatMessage
import torch
from transformers import AutoConfig, AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, TextStreamer
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, AutoModelForSeq2SeqLM
from langchain.llms import HuggingFacePipeline
from sentence_transformers import SentenceTransformer
from langchain.embeddings import HuggingFaceEmbeddings
from langchain import PromptTemplate, LLMChain

/root/miniconda3/envs/cit/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# mistralai/Mistral-7B-Instruct-v0.2
model_name_or_path = "NousResearch/Hermes-2-Pro-Llama-3-8B"

config = AutoConfig.from_pretrained(model_name_or_path, trust_remote_code=True)
config.max_position_embeddings = 8096
quantization_config = BitsAndBytesConfig(
llm_int8_enable_fp32_cpu_offload=True,
bnb_4bit_quant_type='nf4',
bnb_4bit_use_double_quant=True,
bnb_4bit_compute_dtype=torch.bfloat16,
load_in_4bit=True
)

tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=True)
model = AutoModelForCausalLM.from_pretrained(
model_name_or_path,
config=config,
trust_remote_code=True,
quantization_config=quantization_config,
device_map="cuda",
)

# tokenizer = AutoTokenizer.from_pretrained('mistralai/Mistral-7B-Instruct-v0.2', token=hf_token)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Loading checkpoint shards: 100%|██████████| 4/4 [00:26<00:00,  6.58s/it]


In [12]:
!pwd

/root/Projects/citizens-info/notebooks


In [27]:
from dotenv import load_dotenv
import os

current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)
env_dir = os.path.join(parent_dir, 'app')
env_file = os.path.join(env_dir, '.env')

load_dotenv(dotenv_path=env_file)
llama_parse_key = os.getenv("LLAMA_INDEX_TOKEN")

In [31]:
from llama_parse import LlamaParse

parser = LlamaParse(
    result_type = 'markdown',
    verbose = True,
    language = 'en',
    num_workers = 8,
    api_key = llama_parse_key
)

In [38]:
import nest_asyncio

nest_asyncio.apply()

parent_dir = os.path.dirname(current_dir)
path_to_pdf = 'app/pdf_docs'
file_path = os.path.join(parent_dir, path_to_pdf)

c = 0
documents_list = []

for doc in os.listdir(file_path):
    if doc.endswith(".pdf"):
        documents_list.append(f'{file_path}/{doc}')

    c += 1
    if c >= 5:
        break

documents = parser.load_data(documents_list)

Parsing files: 100%|██████████| 5/5 [00:12<00:00,  2.42s/it]


In [39]:
documents

[Document(id_='0a0a0513-e9e8-431b-a2a8-dc47a2276840', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text='# Noise pollution and noise complaints\n\nWhat is noise pollution?\n\n- Is considered a nuisance\n- Is dangerous to human health\n- Could damage property\n- Could damage the environment\n\nThere are no universal noise standard laws that apply in Ireland but your local authority may refer to particular noise standards. Noise pollution is outlined in the Environmental Protection Agency Act 1992.\n\nWho do I complain to about noise pollution?\n\nWhile the law does not mention an exact level or standard of noise that is illegal, if noise is affecting your quality of life, then you have a right to complain. Who you complain to depends on the source of the noise.\n\nYour first step should generally be to contact the person causing the noise to try to fix the issue. If this doesn’t work, you can take the necessary next steps

In [41]:
print(documents[0].text)

# Noise pollution and noise complaints

What is noise pollution?

- Is considered a nuisance
- Is dangerous to human health
- Could damage property
- Could damage the environment

There are no universal noise standard laws that apply in Ireland but your local authority may refer to particular noise standards. Noise pollution is outlined in the Environmental Protection Agency Act 1992.

Who do I complain to about noise pollution?

While the law does not mention an exact level or standard of noise that is illegal, if noise is affecting your quality of life, then you have a right to complain. Who you complain to depends on the source of the noise.

Your first step should generally be to contact the person causing the noise to try to fix the issue. If this doesn’t work, you can take the necessary next steps to make your complaint. There are different bodies who deal with these complaints, but the 2 main places to make your complaints are:

- Local authorities
- The District Court

# Local 

In [3]:
from langchain_community.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader
import os

current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)
path_to_pdf = 'app/pdf_docs'
file_path = os.path.join(parent_dir, path_to_pdf)

documents_text = []
c = 0

for doc in os.listdir(file_path):

    doc_path = os.path.join(file_path, doc)
    loader = PyPDFLoader(doc_path)
    pages = loader.load_and_split()
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 200)
    docs = text_splitter.split_documents(pages)
    documents_text.append(docs)
    c += 1
    if c == 5:
        break

documents_text = [item for sublist in documents_text for item in sublist]

In [6]:
print(documents_text[3].page_content)

You can get a complaint form in your local district court office and will have to pay a fee
to process the complaint.
Other organisations for noise complaints
Depending on the source of the noise, you may complain elsewhere. Other places where
you can lodge a complaint include:
An Garda Síochánna
The Environmental Protection Agency (EPA)
Residential Tenancies Board (RTB)
Transport Infrastructure Ireland (TII)
You can read the gov.ie Guide to Noise Regulations  on the steps you can take if you are
experiencing a nuisance caused by noise.
How to make a noise complaint
If you plan to complain about excessive noise, try to keep a detailed list of:
The times when the noise occurred
How long the noise lasted
The levels of noise involved (if possible)
Step 1: Try to resolve the issue yourself
You should first contact the person or business causing the noise, explain that it is a
nuisance and try to solve the issue together.
Step 2: Contact your local authority or the EPA


In [7]:
pipe  = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_length=250
)

local_llm = HuggingFacePipeline(pipeline=pipe)

template = """can you tell me a joke about {topic}?"""
prompt = PromptTemplate(template=template, input_variables=["spaniards"])
llm_chain = LLMChain(prompt=prompt, llm=local_llm, verbose=True)

model.eval()
with torch.no_grad():
    print(llm_chain.run({"topic": "spaniards"}))


Setting `pad_token_id` to `eos_token_id`:128003 for open-end generation.




> Entering new LLMChain chain...
Prompt after formatting:
can you tell me a joke about spaniards?

> Finished chain.
can you tell me a joke about spaniards? "
A Spaniard walks into a bar and orders a drink. The bartender asks, "Is there a problem?" The Spaniard replies, "No, everything's fine. I just have a little problem with my hearing. In Spain, we call this 'un poco de sordera'." The bartender laughs and says, "Well, in America, we call that a 'little white lie'." 

Here are some funny Spanish jokes:

1. Why did the Spaniard cross the road?
To get to the restaurant on the other side.

2. Why did the Spaniard break up with his girlfriend?
Because she was "un poco loca" (a little crazy).

3. What do you call a Spaniard who speaks three languages?
Un problema (a problem).

4. Why did the Spaniard go to the hospital?
To have his "hijo" (son) removed. 

5. How do you stop a Spaniard from speaking?
With a gun in his mouth.

6. What do you call a Spaniard who can't speak?
Un mudo (a mut